## Лабораторна робота 6

### Виконано студенкою групи АнД-41
### Бовсуновською Марією

#### Завдання
- Знайти в інтернеті матеріал за напрямком дисципліни, що не розглядався на лекціях
- Навести приклад розв'язання відповідного завдання
- ідготувати презентацію матеріалу
- Відповіді оформити .ipynb, .csv(або інші файли з даними), .ppt документами
- Викласти у зазначений репозіторій GitHub в окремій папці з назвою Lab6 де мають бути .ipynb, .ppt файли та необхідні файли з даними
- Підготуватися до захисту та доповіді результатів (окрема доповідь від кожного студента)


### Встановлюємо бібліотеки

In [10]:

%%capture
!apt-get update
!pip install pyglet
!pip install gym pyvirtualdisplay
!pip install xvfbwrapper
!apt-get install -y xvfb python-opengl ffmpeg
!pip install tensorflow==2.1.* 
import gym
from gym.wrappers import Monitor
from collections import deque
import tensorflow as tf
import numpy as np
import random
import math
import time
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [11]:
# Функція на демонстрацію відео
def show_video():
  """Enables video recording of gym environment and shows it."""
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Video not found")
    
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

### Створюємо агента та середовище для демонстрації Deep Q-Network

In [16]:
# Завантажуємо gym environment і отримуємо простори дій та станів та кількість їх елементів.
name_of_environment='CartPole-v0'
env = gym.make(name_of_environment)
num_features = env.observation_space.shape[0]
num_actions = env.action_space.n
print('Number of state features: {}'.format(num_features))
print('Number of possible actions: {}'.format(num_actions))

Number of state features: 4
Number of possible actions: 2


Палиця прикріплена шарніром до візка, котрий рухається по колії без тертя. Управління системою здійснюється шляхом застосування до візка сили +1 або -1. Маятник починається із вертикального положення, мета полягає в тому, щоб запобігти його падінню. Нагорода +1 надається за кожен крок, коли стовп залишається у вертикальному положенні, гра завершується, якщо стовп знаходиться більше ніж на 15 градусів від вертикалі або візок рухається більше ніж на 2,4 одиниці від центру.

Стани які ми запам'ятовуємо - положення візка, швидкість візка, кут жердини, швидкість обертання жердини

Можливі дії агента-візка - сунутися наліво, сунутися направо

Середовище обмежене 200 тіками, тому максимум винагороди, яку можна накопичити - 200

### демонстрація середовища без навчання

In [17]:
# демонстрація середовища без навчання
env_rand = wrap_env(env)
env_rand.reset()
for _ in range(200):
    env_rand.render()
    state, reward, done, info = env_rand.step(env_rand.action_space.sample()) # random action
    if done:
        print("Episode finished after {} timesteps".format(_+1))
        break
env_rand.close()
show_video()

Episode finished after 22 timesteps


### Створюємо додаткові функції

In [18]:
class DQN(tf.keras.Model):
  """Dense neural network class."""
  def __init__(self):
    super(DQN, self).__init__()
    self.dense1 = tf.keras.layers.Dense(32, activation="relu")
    self.dense2 = tf.keras.layers.Dense(32, activation="relu")
    # два виходи у випадку pole
    # без активаційної функції
    self.dense3 = tf.keras.layers.Dense(num_actions, dtype=tf.float32)
    
  def call(self, x):
    """Forward pass."""
    x = self.dense1(x)
    x = self.dense2(x)
    return self.dense3(x)

# створюємо дві нейронні мережі, target буде копією main, але з власною копією ваг. 
main_nn = DQN()
target_nn = DQN()
# визначаємо оптимізатор та loss
optimizer = tf.keras.optimizers.Adam(1e-4)
mse = tf.keras.losses.MeanSquaredError()

In [20]:
class ReplayBuffer(object):
  def __init__(self, size):
    # оголошуємо двусторонню чергу
    self.buffer = deque(maxlen=size)

  def add(self, state, action, reward, next_state, done):
    # метод на додавання у кінець кортежа з новими значеннями.
    # інформація на початку buffer при перевищенні maxlen витісняється
    self.buffer.append((state, action, reward, next_state, done))

  def __len__(self):
    return len(self.buffer)

  def sample(self, num_samples):
    # використовуємо буфер для того щоб отримати буферу у вигляді масивів
    states, actions, rewards, next_states, dones = [], [], [], [], []
    idx = np.random.choice(len(self.buffer), num_samples)
    for i in idx:
      elem = self.buffer[i]
      state, action, reward, next_state, done = elem
      states.append(np.array(state, copy=False))
      actions.append(np.array(action, copy=False))
      rewards.append(reward)
      next_states.append(np.array(next_state, copy=False))
      dones.append(done)
    states = np.array(states)
    actions = np.array(actions)
    rewards = np.array(rewards, dtype=np.float32)
    next_states = np.array(next_states)
    dones = np.array(dones, dtype=np.float32)
    return states, actions, rewards, next_states, dones

In [21]:
def select_epsilon_greedy_action(state, epsilon):
  # рандомний крок якщо менше epsilon, якщо ні, то кращий крок за мережею main
  result = tf.random.uniform((1,))
  if result < epsilon:
    return env.action_space.sample() # Random action (left or right) in pole env.
  else:
    return tf.argmax(main_nn(state)[0]).numpy() # Greedy action for state.

In [23]:
@tf.function
def train_step(states, actions, rewards, next_states, dones):
  # для навчання (однієї ітерації навчання) main мережі за допомогою batch даних, що зберігаються в буфері

  # Calculate targets.
  next_qs = target_nn(next_states)
  # Обчислення tf.math.maximum елементів за розмірами тензора
  max_next_qs = tf.reduce_max(next_qs, axis=-1)
  # y для градієнта, цільова функція 
  target = rewards + (1. - dones) * discount * max_next_qs
  with tf.GradientTape() as tape:
    qs = main_nn(states)
    action_masks = tf.one_hot(actions, num_actions)
    masked_qs = tf.reduce_sum(action_masks * qs, axis=-1)
    # обраховуємо loss
    loss = mse(target, masked_qs)
  # обраховуємо градієнт-похідну та оптимізуємо main
  grads = tape.gradient(loss, main_nn.trainable_variables)
  optimizer.apply_gradients(zip(grads, main_nn.trainable_variables))
  return loss

### Запускаємо навчатися алгоритм DQN.

Навчатимемо основну мережу після кожного третього пробного кроку, щоб не витрачати багато часу.

Імовірність зробити випадковий крок задана спочатку як гарантована, але з кожною епохою ця фмовірність буде зменшуватися (змінна epsilon).

In [26]:
# Hyperparameters.
num_episodes = 1000
epsilon = 1.0
batch_size = 32
# параметр жадності - згоди з новим значенням q
discount = 0.99
buffer = ReplayBuffer(100000)
cur_frame = 0
block_time_start=0

# Start training. Play game once and then train with a batch.
last_100_ep_rewards = []
for episode in range(num_episodes+1):
  if episode % 101 == 0:
    block_time_start=time.time()
  state = env.reset()
  ep_reward, done = 0, False
  while not done:
    state_in = tf.expand_dims(state, axis=0)
    action = select_epsilon_greedy_action(state_in, epsilon)
    next_state, reward, done, info = env.step(action)
    ep_reward += reward
    # Save to experience replay.
    buffer.add(state, action, reward, next_state, done)
    state = next_state
    cur_frame += 1
    # Copy main_nn weights to target_nn.
    if cur_frame % 2000 == 0:
      target_nn.set_weights(main_nn.get_weights())

    # Train neural network.
    if episode % 3 == 0:
      if len(buffer) >= batch_size:
        states, actions, rewards, next_states, dones = buffer.sample(batch_size)
        loss = train_step(states, actions, rewards, next_states, dones)
  
  # поступово знижуємо ймовірність випадкових рухів
  if episode < 950:
    epsilon -= 0.001

  if len(last_100_ep_rewards) == 100:
    last_100_ep_rewards = last_100_ep_rewards[1:]
  last_100_ep_rewards.append(ep_reward)
    
  if episode % 100 == 0:
    block_time=time.time()-block_time_start
    print(f'Episode {episode}/{num_episodes}. Epsilon: {epsilon:.3f}. Time for block: {block_time:.0f}s. '
          f'Reward in last 100 episodes: {np.mean(last_100_ep_rewards):.2f}')
env.close()

Episode 0/1000. Epsilon: 0.999. Time for block: 0s. Reward in last 100 episodes: 36.00
Episode 100/1000. Epsilon: 0.899. Time for block: 2s. Reward in last 100 episodes: 23.34
Episode 200/1000. Epsilon: 0.799. Time for block: 2s. Reward in last 100 episodes: 25.05
Episode 300/1000. Epsilon: 0.699. Time for block: 3s. Reward in last 100 episodes: 33.25
Episode 400/1000. Epsilon: 0.599. Time for block: 6s. Reward in last 100 episodes: 47.03
Episode 500/1000. Epsilon: 0.499. Time for block: 10s. Reward in last 100 episodes: 69.94
Episode 600/1000. Epsilon: 0.399. Time for block: 17s. Reward in last 100 episodes: 109.86
Episode 700/1000. Epsilon: 0.299. Time for block: 28s. Reward in last 100 episodes: 171.02
Episode 800/1000. Epsilon: 0.199. Time for block: 34s. Reward in last 100 episodes: 193.63
Episode 900/1000. Epsilon: 0.099. Time for block: 37s. Reward in last 100 episodes: 198.85
Episode 1000/1000. Epsilon: 0.050. Time for block: 40s. Reward in last 100 episodes: 199.65


### Демонстрація навченого DQN агента у середовищі маятника на візку після тисячі епох.

In [27]:
env = wrap_env(gym.make('CartPole-v0'))
state = env.reset()
done = False
ep_rew = 0
while not done:
  env.render()
  state = tf.expand_dims(state, axis=0)
  action = select_epsilon_greedy_action(state, epsilon=0.01)
  state, reward, done, info = env.step(action)
  ep_rew += reward
print('Нагорода до завершення {}'.format(ep_rew))
env.close()
show_video()

Нагорода до завершення 200.0
